In [1]:
%matplotlib inline

import os
import time
import copy
import pickle
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import h5py
from tqdm.auto import tqdm, trange
import umap
from sampler import ImbalancedDatasetSampler
from ood_metrics import auroc, plot_roc, plot_barcode, calc_metrics, detection_error, fpr_at_95_tpr
import mc_uncertainty as mc

import torch
from torch.nn import functional as F
from torchvision import transforms
import torchvision.models as models
from torchvision.datasets import CIFAR10, CIFAR100, ImageFolder, SVHN
from torch import nn, optim
import torch.utils.data
import torchvision

from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve


CIFAR10_CLASSES = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
FG_CLASSES = list(map(CIFAR10_CLASSES.index, 
                 ["cat"]))
BG_CLASSES = list(map(CIFAR10_CLASSES.index, 
                 ["airplane", "automobile", "bird", "deer", "frog", "dog", "horse", "ship", "truck"]))

NUM_CLASSES = 2
INPUT_SIZE = 224
# INPUT_SIZE = 32
BATCH_SIZE = 64
NUM_FEATURE_LAYERS = 31

RESULTS_FILE = 'cifar10_2class_polarity.h5'

TRAIN = True

In [2]:
def create_model(kind='resnet152'):
    # Download pretrained model
    if kind == 'resnet152':
        model = models.resnet152(pretrained=True)
    elif kind == "densenet161":
        model = models.densenet161(pretrained=True)
    elif kind == "vgg16":
        model = models.vgg16(pretrained=True)
        
    # Freeze weights
    for param in model.parameters():
        param.requires_grad = False

    # Re-implement final classification layer
    if kind == 'resnet152':
        model.fc = nn.Linear(in_features=model.fc.in_features, out_features=NUM_CLASSES, bias=True)
    elif kind == "densenet161":
        model.classifier = nn.Linear(in_features=model.classifier.in_features, out_features=NUM_CLASSES, bias=True)
    elif kind == "vgg16":
        model.classifier[6] = nn.Linear(in_features=model.classifier[6].in_features, out_features=NUM_CLASSES, bias=True)

    return model


class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
    """

    def __init__(self, dataset, indices=None, num_samples=None):
                
        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset))) \
            if indices is None else indices
            
        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples
            
        # distribution of classes in the dataset 
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1
                
        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):
        return dataset.target_transform(dataset.targets[idx])
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples
        


def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, lr_scheduler=None):
    since = time.time()

    train_acc_history = []
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in trange(num_epochs, desc="Epochs"):
        tqdm.write('Epoch {}/{}'.format(epoch, num_epochs - 1))
        tqdm.write('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase], 
                                       total=len(dataloaders[phase]), 
                                       desc="{} batches".format(phase),
                                       leave=False):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            tqdm.write('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_loss = epoch_loss
                val_acc_history.append(epoch_acc)
            elif phase == 'train':
                train_acc_history.append(epoch_acc)
                
        if lr_scheduler is not None:
            lr_scheduler.step(val_loss)
            
        tqdm.write("\n")

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, train_acc_history


def save_model(model, path='model.pt'):
    torch.save(model.state_dict(), path)

    
def load_model(model, path):
    model.load_state_dict(torch.load(path))
    model.eval()


def plot_training_hist(train_hist, val_hist):
    thist = []
    vhist = []

    thist = [h.cpu().numpy() for h in train_hist]
    vhist = [h.cpu().numpy() for h in val_hist]

    plt.title("Accuracy vs. Number of Training Epochs")
    plt.xlabel("Training Epochs")
    plt.ylabel("Accuracy")
    plt.plot(range(1,NUM_EPOCHS+1),thist,label="Train")
    plt.plot(range(1,NUM_EPOCHS+1),vhist,label="Val")
    plt.legend()
    plt.show()
    
    
def save_history(train_hist, val_hist, train_hist_path='train_hist.pickle', val_hist_path='val_hist.pickle'):
    with open(train_hist_path, 'wb') as f:
        pickle.dump(train_hist, f)
    with open(val_hist_path, 'wb') as f:
        pickle.dump(val_hist, f)
        
        
def load_history(train_hist_path='train_hist.pickle', val_hist_path='val_hist.pickle'):
    with open(train_hist_path, 'rb') as f:
        train_hist = pickle.load(f)
    with open(val_hist_path, 'rb') as f:
        val_hist = pickle.load(f)
    return train_hist, val_hist


def evaluate_model(model, dataloader):
    running_corrects = 0
    
    for inputs, labels in tqdm(dataloader, total=len(dataloader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

        # statistics
        running_corrects += torch.sum(preds == labels.data)

    accuracy = running_corrects.double() / len(dataloader.dataset)
    return accuracy.detach().cpu().item()

# Implement a pretrained classifer and finetuning for CIFAR 10

## Set up data loaders

Transform the training data as follows:
    - Remove all images with labels OOD_CLASSES
    - Change the labels:
        - 0: image is in FG_CLASSES (i.e it's a cat)
        - 1: image is one of BG_CLASSES (i.e. it's a airplane, automobile, bird, deer, dog, or frog)
        
Transform the test data as follows:
    - Change the labels:
        - 0: image is in FG_CLASSES (i.e it's a cat)
        - 1: image is one of BG_CLASSES (i.e. it's a airplane, automobile, bird, deer, dog, or frog)
        - 2: image is one of OOD_CLASSES (i.e. it's a horse, ship, or truck)

In [3]:
# Set up data loaders
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.RandomResizedCrop(INPUT_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.CenterCrop(INPUT_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

def label_transform(y):
    if y in FG_CLASSES: return 0
    elif y in BG_CLASSES: return 1
    else: return 2

cifar10_train = CIFAR10('/media/tadenoud/DATADisk/datasets/cifar10/', train=True, transform=data_transforms['train'], 
                        target_transform=label_transform)
cifar10_train_no_aug = CIFAR10('/media/tadenoud/DATADisk/datasets/cifar10/', train=True, transform=data_transforms['val'],
                               target_transform=label_transform)
cifar10_test = CIFAR10('/media/tadenoud/DATADisk/datasets/cifar10/', train=False, transform=data_transforms['val'], 
                       target_transform=label_transform)
cifar100_ood = CIFAR100('/media/tadenoud/DATADisk/datasets/cifar100/', train=False, transform=data_transforms['val'],
                       target_transform=lambda y: y+NUM_CLASSES)
imagenet_resize = ImageFolder('/media/tadenoud/DATADisk/datasets/tiny-imagenet-200/test', transform=data_transforms['val'],
                             target_transform=lambda y: y+NUM_CLASSES)
svhn_ood = SVHN('/media/tadenoud/DATADisk/datasets/svhn', split='test', transform=data_transforms['val'],
                target_transform=lambda y: y+NUM_CLASSES)


dataloaders_dict = {
    'train': torch.utils.data.DataLoader(cifar10_train, batch_size=BATCH_SIZE, 
                                         sampler=ImbalancedDatasetSampler(cifar10_train),
                                         num_workers=4, pin_memory=True),
    'train_no_aug': torch.utils.data.DataLoader(cifar10_train_no_aug, batch_size=BATCH_SIZE, num_workers=4, pin_memory=True),
    'val': torch.utils.data.DataLoader(cifar10_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True),
    'ood': torch.utils.data.DataLoader(cifar100_ood, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True),
    'imagenet_resize': torch.utils.data.DataLoader(imagenet_resize, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True),
    'svhn': torch.utils.data.DataLoader(svhn_ood, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True),
}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
model = create_model("vgg16")
model = model.to(device)

## Define optimization objective, optimizer, etc

In [8]:
class PolarityLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, beta=1, logits=False, reduce=True, num_labels=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.beta = beta
        self.logits = logits
        self.reduce = reduce
        self.num_labels = num_labels
        
    def fp(self, inputs, targets):
        pl = inputs[:,targets].diag()
        delta_p = torch.sub(inputs, pl.unsqueeze(dim=1))
        return F.sigmoid(self.beta * (delta_p))
        
    def focal_loss(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, F.one_hot(targets, self.num_labels).float(), reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, F.one_hot(targets, self.num_labels).float(), reduce=False)
        pt = torch.exp(-BCE_loss)
        return self.alpha * (1-pt)**self.gamma * BCE_loss
    
    def forward(self, inputs, targets):
        p_loss = self.fp(inputs, targets)
        f_loss = self.focal_loss(inputs, targets)
        loss = p_loss.mm(f_loss.transpose(0,1)).diag()
        
        if self.reduce:
            return torch.mean(loss)
        else:
            return loss

In [9]:
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model.parameters()
print("Params to learn:")

params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
scheduler_ft = optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft, patience=5)

# Setup the loss fxn
criterion = PolarityLoss(alpha=0.25, gamma=2, beta=20, logits=True, reduce=True)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


## Train and evaluate

In [10]:
NUM_EPOCHS = 20
if TRAIN:
    # Train and evaluate
    model, val_hist, train_hist = train_model(
        model, 
        dataloaders_dict, 
        criterion, 
        optimizer_ft, 
        num_epochs=NUM_EPOCHS,
        lr_scheduler=scheduler_ft)
    save_model(model, path='vgg16_polarity.pt')
    save_history(train_hist, val_hist, 'train_hist_polarity.pickle', 'val_hist_polarity.pickle')
else:
    load_model(model, path='vgg16_polarity.pt')
    train_hist, val_hist = load_history('train_hist_polarity.pickle', 'val_hist_polarity.pickle')

plot_training_hist(train_hist, val_hist)    

Epoch 0/19
----------


train Loss: 0.0319 Acc: 0.7638


val Loss: 0.0190 Acc: 0.8653


Epoch 1/19
----------


train Loss: 0.0286 Acc: 0.7869


val Loss: 0.0198 Acc: 0.8585


Epoch 2/19
----------


...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



val Loss: 0.0150 Acc: 0.8918


Epoch 18/19
----------


train Loss: 0.0245 Acc: 0.8148


val Loss: 0.0156 Acc: 0.8856


Epoch 19/19
----------


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Fine tune top convolutional layers

In [11]:
# print(model)

# Freeze weights
for param in model.parameters():
    param.requires_grad = False
    
# Unfreeze top layers
for param in model.features[23:].parameters():
    param.requires_grad = True
    
for param in model.classifier.parameters():
    param.requires_grad = True

In [27]:
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model.parameters()
print("Params to learn:")

params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
        
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.0001, momentum=0.9)
scheduler_ft = optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft, patience=5)

# Setup the loss fxn
criterion = PolarityLoss(alpha=0.25, gamma=2, beta=20, logits=True, reduce=True)

Params to learn:
	 features.24.weight
	 features.24.bias
	 features.26.weight
	 features.26.bias
	 features.28.weight
	 features.28.bias
	 classifier.0.weight
	 classifier.0.bias
	 classifier.3.weight
	 classifier.3.bias
	 classifier.6.weight
	 classifier.6.bias


In [ ]:
NUM_EPOCHS = 50
if TRAIN:
    # Train and evaluate
    model, val_hist, train_hist = train_model(model, 
                                              dataloaders_dict, 
                                              criterion, 
                                              optimizer_ft, 
                                              num_epochs=NUM_EPOCHS,
                                              lr_scheduler=scheduler_ft)
    save_model(model, path='vgg16_fine_polarity.pt')
    save_history(train_hist, val_hist, 'train_hist_fine_polarity.pickle', 'val_hist_fine_polarity.pickle')
    
else:
    load_model(model, path='vgg16_fine_polarity.pt')
    train_hist, val_hist = load_history('train_hist_fine_polarity.pickle', 'val_hist_fine_polarity.pickle')

plot_training_hist(train_hist, val_hist)

Epoch 0/49
----------


/home/tadenoud/anaconda3/envs/nn/lib/python3.6/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/tadenoud/anaconda3/envs/nn/lib/python3.6/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


train Loss: 0.0243 Acc: 0.8166


val Loss: 0.0142 Acc: 0.8980


Epoch 1/49
----------


train Loss: 0.0241 Acc: 0.8198


...

val Loss: 0.0146 Acc: 0.8948


Epoch 25/49
----------


train Loss: 0.0233 Acc: 0.8250


val Loss: 0.0146 Acc: 0.8948


Epoch 26/49
----------


train Loss: 0.0232 Acc: 0.8259


val Loss: 0.0146 Acc: 0.8948


Epoch 27/49
----------


In [ ]:
print("Model accuracy:", evaluate_model(model, dataloaders_dict['val'])) 

# Train Mahalanobis Distance function

In [ ]:
def penultimate_forward_vgg16(self, x):
    x = self.features(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    
    # Break up self.classifier
    penultimate = model.classifier[:5](x)
    out = model.classifier[5:](penultimate)
    return out, penultimate

# Bind a function to the model to extract penultimate features
model.penultimate_forward = penultimate_forward_vgg16.__get__(model)

In [ ]:
def extract_features(model, dataloader, feature_size, num_classes):
    all_outputs = np.empty((0,num_classes))
    all_features = np.empty((0,feature_size))
    all_labels = np.empty(0)

    model.eval()
    for X, Y in tqdm(dataloader, total=len(dataloader)):
        X = X.to(device)

        outputs, features = model.penultimate_forward(X)

        features = features.view((features.size(0), features.size(1), -1))
        features = torch.mean(features, 2)

        features = features.detach().cpu().numpy()

        all_features = np.concatenate((all_features, features), axis=0)
        all_outputs = np.concatenate((all_outputs, outputs.detach().cpu().numpy()), axis=0)
        all_labels = np.concatenate((all_labels, Y.detach().cpu().numpy()), axis=0)
        
    return all_outputs, all_features, all_labels

In [ ]:
# Get penultimate features, outputs, and labels from datasets set
if TRAIN:
    model.eval()
    
    for dataset in ['train_no_aug', 'val', 'ood', 'svhn']:
        softmax_outputs, mahalanobis_features, labels_out = extract_features(model, dataloaders_dict[dataset], feature_size=4096, num_classes=NUM_CLASSES)

        # Save outputs to disk
        with h5py.File(RESULTS_FILE, "a") as f:
            if dataset in f.keys():
                del f[dataset]
                
            g = f.create_group(dataset)
            g.create_dataset("softmax_outputs", data=softmax_outputs)
            g.create_dataset("mahalanobis_features", data=mahalanobis_features)
            g.create_dataset("labels", data=labels_out)

In [ ]:
import sklearn.covariance

class Mahal_Dist(object):
    def __init__(self, gt_features=None, gt_labels=None):
        super().__init__()
        
        if gt_features is not None and gt_labels is not None:
            self.fit(gt_features, gt_labels)
        
    def fit(self, gt_features, gt_labels, num_classes):
        """Fit parameters used for Mahalanobis distance."""
        self.class_means = []
        centered_features = gt_features.copy()

        for i in range(num_classes):
            mask = gt_labels == i

            # Calculate the class mean
            self.class_means.append(gt_features[mask].mean(axis=0))

            # Create mean subtracted features for covariance estimation
            centered_features[mask] -= self.class_means[i]


        # Calculate the inverted covariance matrix
        group_lasso = sklearn.covariance.EmpiricalCovariance(assume_centered=False)
        group_lasso.fit(centered_features)
        self.precision = group_lasso.precision_
        self.covariance = group_lasso.covariance_

        # Convert to pytorch tensors
        self.class_means = torch.from_numpy(np.array(self.class_means)).float().to(device)
        self.precision = torch.from_numpy(self.precision).float().to(device)
        self.covariance = torch.from_numpy(self.covariance).float().to(device)
        
        
    def __call__(self, features):
        """Return the distance based confidence score to each 
            of the means."""
        # Tile the features and means to vectorize the operation
        num_means = self.class_means.shape[0]
        num_feats = features.shape[0]

        features = features.repeat(1, num_means).view(-1, features.shape[1])

        # subtract means in batches
        x = features - self.class_means.repeat(num_feats, 1)

        # matmul, take diagonal, then reshape
        dists = x.mm(self.precision).mm(x.transpose(0,1)).diag()
        dists = dists.view(num_feats, num_means).transpose(0,1)

        return dists
    
    
    def label_dist(self, features):
        """Return the label of the closest mean and the 
            mahalanobis distance to it."""
        dists = self(features)
        min_dists, labels = dists.min(dim=0)
        return labels, min_dists 
    
    
    def save(self, fname='mahal_distance.h5'):
        """Save precomputed Mahal_Dist object with all means and covariance matrix."""
        with h5py.File(fname, 'a') as f:
            # Remove old data if exists
            if "mahal_distance" in f.keys():
                del f['mahal_distance']
                
            # Store the new data
            g = f.create_group("mahal_distance")
            g.create_dataset("means", data=self.class_means.detach().cpu().numpy())
            g.create_dataset("cov", data=self.covariance.detach().cpu().numpy())
            g.create_dataset("inv_cov", data=self.precision.detach().cpu().numpy())
           
        
    @classmethod
    def load(cls, fname='mahal_distance.h5', device=torch.device("cpu")):
        """Load precomputed Mahal_Dist object with all means and covariance matrix."""
        inst = cls()
        
        with h5py.File(fname, 'r') as f:
            inst.class_means = torch.Tensor(f['mahal_distance/means'].value).to(device)
            inst.covariance = torch.Tensor(f['mahal_distance/cov'].value).to(device)
            inst.precision = torch.Tensor(f['mahal_distance/inv_cov'].value).to(device)
        
        return inst

In [ ]:
if TRAIN:  
    with h5py.File(RESULTS_FILE, "r") as f:
        mahalanobis_features_train = f['train_no_aug/mahalanobis_features'].value
        labels_train = f['train_no_aug/labels'].value
    
    md = Mahal_Dist()
    md.fit(mahalanobis_features_train, labels_train, num_classes=NUM_CLASSES)
    md.save(RESULTS_FILE)
else:
    md = Mahal_Dist.load(RESULTS_FILE, device)

# Extract FG, BG, OOD Softmax predictions and Mahal features

In [ ]:
from torch import Tensor

if TRAIN:
    for dataset in ['val', 'ood', 'svhn']:
        print(dataset)
        with h5py.File(RESULTS_FILE, 'r') as f:
            g = f[dataset]
            labels = g['labels'][()]
            features = g['mahalanobis_features'][()]
            outputs = g['softmax_outputs'][()]    
        
        # Get predicted labels, distance to nearest mean
        cur = 0
        mahalanobis_labels = np.empty(0,)
        mahalanobis_dists = np.empty(0,)
        while cur < len(features):
            l, d = md.label_dist(Tensor(features[cur:cur+BATCH_SIZE]).to(device))
            mahalanobis_labels = np.concatenate((mahalanobis_labels, l.detach().cpu().numpy()), axis=0)
            mahalanobis_dists = np.concatenate((mahalanobis_dists, d.detach().cpu().numpy()), axis=0)
            cur += BATCH_SIZE
        
        with h5py.File(RESULTS_FILE, 'a') as f:
            g = f[dataset]
            if 'mahalanobis_labels' in g.keys():
                del g['mahalanobis_labels']
            g.create_dataset('mahalanobis_labels', data=mahalanobis_labels)
            if 'mahalanobis_dists' in g.keys():
                del g['mahalanobis_dists']
            g.create_dataset('mahalanobis_dists', data=mahalanobis_dists)    

In [ ]:
with h5py.File(RESULTS_FILE, 'r') as f:
    g = f['val']
    labels_val = g['labels'][()]
    mahalanobis_labels_val = g['mahalanobis_labels'][()]
    mahalanobis_dist_val = g['mahalanobis_dists'][()]

correct = np.sum((mahalanobis_labels_val == labels_val).astype(np.int))
print("%d%% of val data classified correctly with mahalanobis distance" % (correct / len(labels_val) * 100))

# Regular Mahalanobis to separate CIFAR10 and CIFAR100

In [ ]:
with h5py.File(RESULTS_FILE, "r") as f:
    mahalanobis_dist_val = f['val/mahalanobis_dists'].value
    print(mahalanobis_dist_val.shape)
    labels_val = f['val/labels'].value

    mahalanobis_dist_ood = f['ood/mahalanobis_dists'].value
    print(mahalanobis_dist_ood.shape)
    labels_ood = f['ood/labels'].value

scores = np.concatenate((mahalanobis_dist_val, mahalanobis_dist_ood), axis=0)
labels = np.concatenate((np.zeros(len(mahalanobis_dist_val)), np.ones(len(mahalanobis_dist_ood))), axis=0)

plot_roc(scores, labels)

# Problem statement

In [ ]:
from umap import UMAP

with h5py.File(RESULTS_FILE, 'r') as f:
    features_train = f['train_no_aug/mahalanobis_features'].value
    labels_train = f['train_no_aug/labels'].value
    
    features_val = f['val/mahalanobis_features'].value
    labels_val = f['val/labels'].value + 2
    
    features_ood = f['ood/mahalanobis_features'].value
    labels_ood = np.ones(len(features_ood)) * 4
    
features = np.concatenate((features_train, features_val, features_ood), axis=0)
labels = np.concatenate((labels_train, labels_val, labels_ood), axis=0).astype(np.int)

In [ ]:
embedding = UMAP(n_neighbors=30).fit_transform(features)

In [ ]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

fig, ax = plt.subplots(figsize=(10, 5))
plt.title("Train data and Val data")
text_labels = ['FG_train', 'BG_train', 'FG_val', 'BG_val']

for label in range(len(text_labels)):
    x = embedding[:,0][labels == label]
    y = embedding[:,1][labels == label]
    ax.scatter(x, y, c=colors[label], s=5, label=text_labels[label], alpha=0.8, edgecolors='none')

ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
plt.title("Train data and Val data")
text_labels = ['FG_train', 'BG_train', 'OOD']

for label in range(len(text_labels)):
    legend_label = text_labels[label]
    if legend_label == 'OOD':
        label = 4
    
    x = embedding[:,0][labels == label]
    y = embedding[:,1][labels == label]
    ax.scatter(x, y, c=colors[label], s=5, label=legend_label, alpha=0.8, edgecolors='none')

ax.legend()
plt.show()

Possible problem solutions

- Cluster inlier classes used for Mahalanobis distance using GMM
    - Problem: How many components? What form of covariance?
    
- Increase interclass distance in feature space using Polarity Loss

- Buu provided paper on Neural Random Forest for OOD detection (https://openreview.net/forum?id=HygTE309t7)